# Custom Action Features

## Overview

### Questions

- How do I access simulation state information?
- How do I create loggable quantities in custom actions?
- What are other features provided by the custom action/operation
  API?

### Objectives

- Explain how to access simulation state in a custom action.
- Explain how to expose loggable quantities in a custom action.
- Demonstrate other miscellaneous features of custom actions.

## Boilerplate Code

In [1]:
import hoomd

def clear_sim(sim):
    """Remove all operations (besides intergrator) from the sim."""
    sim.operations.updaters.clear()
    sim.operations.writers.clear()
    del sim.operations.tuners[1:] # keep the particle sorter

cpu = hoomd.device.CPU()
sim = hoomd.Simulation(cpu)

snap = hoomd.Snapshot()
snap.particles.N = 1
snap.particles.position[:] = [0, 0, 0]
snap.particles.types = ['A']
snap.particles.typeid[:] = [0]
snap.configuration.box = [10, 10, 10, 0, 0, 0]

sim.create_state_from_snapshot(snap)

## How do I access simulation state?

By the time that a custom action will have its `act` method called
it will have an attribute `_state` accessable to it which is the
simulation state for the simulation it is associated with. The behavior
of this is controlled in the `hoomd.custom.Action.attach` method. The
method takes in a simulation object and performs any necessary set-up
for the action call `act`. By default, the method stores the simulation
state in the `_state` attribute.

We will create two custom actions class to show this. In one, we will
not modify the `attach` method, and in the other we will make `attach`
a no-op. In each class we will make the action test for the existence
of the `_state` attribute and print some information about the `_state`
if it exists.

In [2]:
class InspectState(hoomd.custom.Action):
    def act(self, timestep):
        if hasattr(self, '_state'):
            print("Has state.")
            print(f"Type {type(self._state)}, ", end='')
            print(f"Number of particles {self._state.N_particles}.")
        else:
            print("Uh-oh no '_state' to be found.")

class BrokenInspectState(InspectState):
    def attach(self, simulation):
        pass

Like in the previous section these are both writers. We will go ahead
and wrap them and see what happens when we try to run the simulation.

In [3]:
inspect_state = InspectState()
sim.operations.writers.append(
    hoomd.write.CustomWriter(inspect_state,
                             trigger=hoomd.trigger.Periodic(10))
)
sim.run(11)

Has state.
Type <class 'hoomd.state.State'>, Number of particles 1.


In [4]:
clear_sim(sim)
broken_inspect_state = BrokenInspectState()
sim.operations.writers.append(
    hoomd.write.CustomWriter(broken_inspect_state,
                             trigger=hoomd.trigger.Periodic(10))
)
sim.run(11)

Uh-oh no '_state' to be found.


## Loggable Quantities in Custom Actions

Custom actions can hook into HOOMD-blue's logging subsystem by using
the `hoomd.logging.log` decorator to document which methods/properties
of a custom action are loggable. See the documentation on `hoomd.logging.log` and `hoomd.logging.TypeFlags` for complete documenation of the decorator and loggable types.

In general, `log` as a decorator takes optional arguments setting whether to make a method a property `is_property` default `True`, what
type the loggable quantity is `category` default `'scalar'` (see `TypeFlags` for a list of all types), and whether the quantity should be logged
by default.

Rather than elaborate, we will use an example to explain these attributes.

In [5]:
class ActionWithLoggables(hoomd.custom.Action):
    @hoomd.logging.log
    def scalar_property_loggable(self):
        return 42
    
    @hoomd.logging.log(is_property=False)
    def scalar_method_loggable(self, default_arg=3):
        return default_arg / 2
    
    @hoomd.logging.log(default=False)
    def scalar_performance_loggable(self):
        return 1_000
    
    @hoomd.logging.log(category='string')
    def string_loggable(self):
        return "I am a string loggable."
    
    def act(self, timestep):
        pass

action = ActionWithLoggables()
print(action.loggables)
print(action.scalar_property_loggable)
try:
    action.scalar_property_loggable()
except TypeError:
    print("Opps this is a property now.")
print(action.scalar_method_loggable(),
      action.scalar_method_loggable(6))
print(action.string_loggable)

{'scalar_property_loggable': 'scalar', 'scalar_method_loggable': 'scalar', 'scalar_performance_loggable': 'scalar', 'string_loggable': 'string'}
42
Opps this is a property now.
1.5 3.0
I am a string loggable.


## Custom Operation Wrapping

Another feature of the custom action API is that when an object is
wrapped by a custom operation object, the action's attributes are available through the operation object as if the operation were the
action. For example, we will wrap `action` from the previous code block
in a `CustomWriter` and access its attributes that way.

Due to this wrapping the attribute `trigger` should not exist in your
custom action.

In [6]:
custom_op = hoomd.write.CustomWriter(action, 100)
print(custom_op.scalar_property_loggable)
print(custom_op.scalar_method_loggable(),
      custom_op.scalar_method_loggable(6))
print(custom_op.string_loggable)

42
1.5 3.0
I am a string loggable.


## Setting Integrator Flags

HOOMD-blue does not always calculate certain quantities to save on
computation. These include the rotational kinetic energy, the pressure
tensor, and external field virials. If you do not know what one of these
quantities are then you likely do not need to use it for your custom
action. However, if you need one of these quantities to be available
for your custom action, they can be set using either a class level or
object level attribute `flags`. `flags` is expected to be a list or
sequence of the required flags.

Below is an example that requests the pressure tensor be available on
any timestep it is run. (Keep in mind that HOOMD-blue will still not
calculate the pressure tensor for timesteps your action will not run
unless another operation needs the quantity).

In [7]:
class ActionWithIntegratorFlags(hoomd.custom.Action):
    # This is the preferred way of specifying custom action
    # integrator flags
    flags = [hoomd.custom.Action.Flags.PRESSURE_TENSOR]
    
    def act(self, timestep):
        pass

## Summary

These surmise most of the unique features of custom actions in Python.
They are
- Accessing simulation state through `_state`
- Exposing loggable quantities
- Accessing action attributes through custom operation wrapper
- Setting integrator flags

With this information, you could write any action that is possible to
write in Python for use in HOOMD-blue. The remain tutorial sections will
focus on concrete examples and show some tricks to get the best
performance.